# Premier preprocessing

## Préambule

* Importation des librairies

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from tabulate import tabulate
from IPython.display import display, HTML
import seaborn as sns
import matplotlib.pyplot as plt
import csv
import os
import logging
import math
import re
import json

* Configuration de la journalisation

In [6]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## Chargement des données

In [8]:
# Détection du délimiteur utilisé dans chaque fichier CSV
def get_delimiter(file_path, bytes=4096):
    try:
        with open(file_path, 'r') as file:
            data = file.read(bytes)
        sniffer = csv.Sniffer()
        delimiter = sniffer.sniff(data).delimiter
        return delimiter
    except Exception as e:
        logging.error(f"Erreur lors de la détection du délimiteur: {e}")
        return None

# Lecture des fichiers CSV avec différents encodages
def read_csv_file(file_path):
    if not os.path.exists(file_path):
        return None, False, f"Fichier non trouvé: {file_path}"
    
    delimiter = get_delimiter(file_path)
    if not delimiter:
        return None, False, f"Impossible de détecter le délimiteur pour le fichier: {file_path}"
    
    encodings = ['utf-8', 'latin1', 'ISO-8859-1']
    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, low_memory=False, encoding=encoding, delimiter=delimiter)
            return df, True, None
        except (UnicodeDecodeError, pd.errors.ParserError) as e:
            logging.warning(f"Erreur avec l'encodage {encoding} pour le fichier {file_path}: {e}")
    
    return None, False, f"Impossible de lire le fichier {file_path} avec les encodages: {encodings}."

# Chargement des datasets en fonction des préfixes et des années spécifiés
def load_datasets(prefixes, years, base_path='data/raw'):
    dataframes = []
    
    for prefix in prefixes:
        datasets = []
        for year in years:
            # Détermine le connecteur en fonction de l'année
            connector = '_' if year <= 2016 else '-'
            
            # Gestion de deux orthographes pour 'caracteristiques'
            if prefix == 'caracteristiques':
                # Essai avec 'caracteristiques'
                filename = f'caracteristiques{connector}{year}.csv'
                file_path = os.path.join(base_path, filename)
                if not os.path.exists(file_path):
                    # Si pas trouvé, essai avec 'carcteristiques'
                    filename = f'carcteristiques{connector}{year}.csv'
                    file_path = os.path.join(base_path, filename)
            else:
                filename = f'{prefix}{connector}{year}.csv'
                file_path = os.path.join(base_path, filename)
            
            if os.path.exists(file_path):
                df, success, error = read_csv_file(file_path)
                if success:
                    datasets.append({filename: df})
                else:
                    logging.error(error)
                    
        dataframes.append(datasets)
    
    return dataframes

# Spécification des années et des préfixes pour les fichiers CSV
years = list(range(2019, 2023))
prefixes = ['caracteristiques', 'lieux', 'usagers', 'vehicules']

# Chargement des datasets
dataframes = load_datasets(prefixes, years)

# Log des datasets chargés
for prefix, df_list in zip(prefixes, dataframes):
    logging.info(f'{prefix}: {len(df_list)} datasets chargés.')

logging.info(f'Total datasets chargés: {sum(len(dfs) for dfs in dataframes)}.')

2024-10-24 15:51:11,990 - INFO - caracteristiques: 4 datasets chargés.
2024-10-24 15:51:11,990 - INFO - lieux: 4 datasets chargés.
2024-10-24 15:51:12,004 - INFO - usagers: 4 datasets chargés.
2024-10-24 15:51:12,005 - INFO - vehicules: 4 datasets chargés.
2024-10-24 15:51:12,006 - INFO - Total datasets chargés: 16.


## Définition de fonctions spécifiques

In [10]:
# Conversion des types de données
# La fonction convert_dtypes convertit les types de données des colonnes d'un DataFrame selon une structure de référence


def convert_dtypes(df, reference_dtypes):
    # Remplacer toutes les valeurs NA par -1
    df = df.fillna(-1)
    
    for col in df.columns:
        if col in reference_dtypes:
            try:
                df.loc[:, col] = df[col].astype(reference_dtypes[col])  # Utilisation de .loc pour éviter les warnings
            except ValueError as e:
                logging.error(f"Erreur lors de la conversion de la colonne {col} en type {reference_dtypes[col]}: {e}")
    return df

In [11]:
# Extraction de la structure de référence
# La fonction extract_reference_structure extrait les structures de référence (types de données des colonnes) des derniers fichiers 
# chargés pour chaque préfixe.


def extract_reference_structure(dataframes):
    reference_structures = {}
    
    for prefix, df_list in zip(prefixes, dataframes):
        if df_list:
            last_df = list(df_list[-1].values())[0]
            reference_structures[prefix] = last_df.dtypes.to_dict()
    return reference_structures

In [12]:
# Prétraitement des données
# La fonction preprocess effectue diverses opérations de prétraitement sur les DataFrames selon des règles spécifiques à chaque 
# type de fichier (caractéristiques, lieux, usagers, véhicules).
# La fonction preprocess_datasets applique le prétraitement à tous les datasets chargés.


def preprocess(df, prefix):
    if prefix == 'caracteristiques':
        # Renommer 'Accident_Id' en 'Num_Acc' si nécessaire
        if 'Accident_Id' in df.columns:
            df = df.rename(columns={'Accident_Id': 'Num_Acc'})

        # Ajouter 2000 à 'an' si < 2000
        if 'an' in df.columns:
            df.loc[:, 'an'] = df['an'].apply(lambda x: x + 2000 if x < 2000 else x)
        
        # Convertir 'hrmn' de 'HHMM' à 'HH:MM'
        if 'hrmn' in df.columns:
            df.loc[:, 'hrmn'] = df['hrmn'].apply(lambda x: f"{str(x).zfill(4)[:2]}:{str(x).zfill(4)[2:]}")
        
        # Remplacer les valeurs NaN par -1
        for col in ['lum', 'int', 'atm', 'col']:
            if col in df.columns:
                df[col] = df[col].fillna(-1)
        
        # Supprimer les colonnes 'adr', 'lat', 'long'
        df = df.drop(columns=['adr', 'lat', 'long'], errors='ignore')

    elif prefix == 'lieux':
        # Remplacer les valeurs NaN par -1
        for col in ['circ', 'vosp', 'prof', 'pr', 'pr1', 'plan', 'surf', 'infra', 'situ']:
            if col in df.columns:
                df[col] = df[col].fillna(-1)
        
        # Remplacer les valeurs NaN et 0 par -1 pour 'lartpc'
        if 'lartpc' in df.columns:
            df['lartpc'] = df['lartpc'].replace(0, -1).fillna(-1)
        
        # Remplacer les valeurs NaN et > 130 par -1 pour 'vma'
        if 'vma' in df.columns:
            df.loc[df['vma'] > 130, 'vma'] = -1
            df['vma'] = df['vma'].fillna(-1)
            
        # Supprimer les colonnes 'voie', 'v1', 'v2', 'larrout'
        df = df.drop(columns=['voie', 'v1', 'v2', 'larrout'], errors='ignore')

    elif prefix == 'usagers':
        # Remplacer les valeurs NaN par -1
        for col in ['place', 'catu', 'grav', 'sexe', 'trajet', 'secu1', 'secu2', 'secu3', 'locp', 'actp', 'etatp']:
            if col in df.columns:
                df[col] = df[col].fillna(-1)
        
        # Remplacer les valeurs 4 par -1 pour 'catu'
        if 'catu' in df.columns:
            df['catu'] = df['catu'].replace(4, -1)
        
        # Gérer les outliers pour 'an_nais'
        if 'an_nais' in df.columns:
            df['an_nais'] = df['an_nais'].apply(lambda x: pd.NA if x < 1900 else x)
    
    elif prefix == 'vehicules':
        # Remplacer les valeurs NaN par -1
        for col in ['senc', 'obs', 'obsm', 'choc', 'manv', 'motor']:
            if col in df.columns:
                df[col] = df[col].fillna(-1)
        
        # Remplacer les valeurs NaN par 0 pour 'catv'
        if 'catv' in df.columns:
            df['catv'] = df['catv'].fillna(0)
        
        # Supprimer la colonne 'occutc'
        df = df.drop(columns=['occutc'], errors='ignore')
    
        for col in df.columns:
            if df[col].dtype == np.float64 or df[col].dtype == np.int64:
                df.loc[:, col] = df[col].fillna(-1)

        # Gestion des cas particuliers
        if 'lartpc' in df.columns:
            df.loc[:, 'lartpc'] = df['lartpc'].replace(0, -1).fillna(-1)
        if 'catu' in df.columns:
            df.loc[:, 'catu'] = df['catu'].replace(4, -1)
        if 'an_nais' in df.columns:
            df.loc[:, 'an_nais'] = df['an_nais'].apply(lambda x: pd.NA if x < 1900 else x)
        
    return df
    


In [13]:
# Prétraitement des ensembles de données
# La fonction applique 'preprocess' sur tous les datasets


def preprocess_datasets(dataframes):
    for prefix, df_list in zip(prefixes, dataframes):
        for df_dict in df_list:
            for file_name, df in df_dict.items():
                df = preprocess(df, prefix)
                df_dict[file_name] = df
    return dataframes

In [14]:
# Harmonisation des dataframes selon la structure de référence
# La fonction harmonize_dataframes harmonise les DataFrames selon les structures de référence, en conservant uniquement les colonnes 
# présentes dans la structure de référence et en convertissant les types de données.


def harmonize_dataframes(dataframes, reference_structures):
    harmonized_dataframes = []
    
    for prefix, df_list in zip(prefixes, dataframes):
        reference_dtypes = reference_structures.get(prefix, {})
        harmonized_group = []
        
        for df_dict in df_list:
            for file_name, df in df_dict.items():
                # Garder uniquement les colonnes présentes dans la structure de référence
                df = df[[col for col in df.columns if col in reference_dtypes]]
                
                # Convertir les données
                df = convert_dtypes(df, reference_dtypes)
                
                harmonized_group.append({file_name: df})
        
        harmonized_dataframes.append(harmonized_group)
    
    return harmonized_dataframes

In [15]:
# Comparaison des structures réelles avec les structures de référence
# La fonction 'compare_structures' compare les structures des DataFrames harmonisés avec les structures de référence et enregistre les différences.


def compare_structures(harmonized_dataframes, reference_structures):
    for prefix, df_list in zip(prefixes, harmonized_dataframes):
        reference_dtypes = reference_structures.get(prefix, {})
        for df_dict in df_list:
            for file_name, df in df_dict.items():
                df_dtypes = df.dtypes.to_dict()
                missing_cols = set(reference_dtypes.keys()) - set(df_dtypes.keys())
                extra_cols = set(df_dtypes.keys()) - set(reference_dtypes.keys())
                diff_types = {col: (reference_dtypes[col], df_dtypes[col]) for col in df_dtypes if col in reference_dtypes and reference_dtypes[col] != df_dtypes[col]}

                table_data = [
                    ["Colonnes manquantes", ", ".join(missing_cols)],
                    ["Colonnes supplémentaires", ", ".join(extra_cols)],
                    ["Différences de types", ", ".join([f"{col} (réf: {reference_dtypes[col]}, fichier: {df_dtypes[col]})" for col in diff_types])]
                ]
                table = tabulate(table_data, headers=["Type de différence", "Colonnes"], tablefmt="grid")
                
                logging.info(f"Comparaison pour le fichier {file_name}:\n{table}")

In [16]:
# Liste de dataframes concaténés
# La fonction concat_harmonized_dataframes concatène les DataFrames harmonisés de chaque groupe en un seul DataFrame par type.


def concat_harmonized_dataframes(dataframes):
    concatenated_dataframes = []

    for df_list in dataframes:
        
        df_only_list = [list(d.values())[0] for d in df_list]
        concatenated_df = pd.concat(df_only_list, ignore_index=True)
        concatenated_dataframes.append(concatenated_df)
    
    return concatenated_dataframes

In [17]:
# Fusion des dataframes


def merge_dataframes(concatenated_dataframes):
    df_caracteristiques = concatenated_dataframes[0]
    df_lieux = concatenated_dataframes[1]
    df_usagers = concatenated_dataframes[2]
    df_vehicules = concatenated_dataframes[3]
    
    merged_df = pd.merge(df_caracteristiques, df_lieux, on='Num_Acc', how='inner')
    merged_df = pd.merge(merged_df, df_usagers, on='Num_Acc', how='inner')
    merged_df = pd.merge(merged_df, df_vehicules, on=['Num_Acc', 'id_vehicule', 'num_veh'], how='inner')
    
    return merged_df

## Lancement du processus de traitement

In [19]:
dataframes = preprocess_datasets(dataframes)
reference_structures = extract_reference_structure(dataframes)
harmonized_dataframes = harmonize_dataframes(dataframes, reference_structures)

## Contrôles avant fusion

In [21]:
# Vérification des datasets chargés
for prefix, df_list in zip(prefixes, dataframes):
    logging.info(f'{prefix}: {len(df_list)} datasets chargés.')

logging.info(f'Total datasets chargés: {sum(len(dfs) for dfs in dataframes)}.')

2024-10-24 15:51:12,971 - INFO - caracteristiques: 4 datasets chargés.
2024-10-24 15:51:12,972 - INFO - lieux: 4 datasets chargés.
2024-10-24 15:51:12,973 - INFO - usagers: 4 datasets chargés.
2024-10-24 15:51:12,973 - INFO - vehicules: 4 datasets chargés.
2024-10-24 15:51:12,974 - INFO - Total datasets chargés: 16.


In [22]:
# Afficher les structures de référence pour vérification

for prefix, structure in reference_structures.items():
    logging.info(f'Structure de référence pour {prefix}: {structure}')

2024-10-24 15:51:12,983 - INFO - Structure de référence pour caracteristiques: {'Num_Acc': dtype('int64'), 'jour': dtype('int64'), 'mois': dtype('int64'), 'an': dtype('int64'), 'hrmn': dtype('O'), 'lum': dtype('int64'), 'dep': dtype('O'), 'com': dtype('O'), 'agg': dtype('int64'), 'int': dtype('int64'), 'atm': dtype('int64'), 'col': dtype('int64')}
2024-10-24 15:51:12,984 - INFO - Structure de référence pour lieux: {'Num_Acc': dtype('int64'), 'catr': dtype('int64'), 'circ': dtype('int64'), 'nbv': dtype('O'), 'vosp': dtype('int64'), 'prof': dtype('int64'), 'pr': dtype('O'), 'pr1': dtype('O'), 'plan': dtype('int64'), 'lartpc': dtype('O'), 'surf': dtype('int64'), 'infra': dtype('int64'), 'situ': dtype('int64'), 'vma': dtype('int64')}
2024-10-24 15:51:12,987 - INFO - Structure de référence pour usagers: {'Num_Acc': dtype('int64'), 'id_usager': dtype('O'), 'id_vehicule': dtype('O'), 'num_veh': dtype('O'), 'place': dtype('int64'), 'catu': dtype('int64'), 'grav': dtype('int64'), 'sexe': dtype(

In [23]:
# Vérification des datasets harmonisés
for prefix, df_list in zip(prefixes, harmonized_dataframes):
    logging.info(f'{prefix}: {len(df_list)} datasets harmonisés.')

logging.info(f'Total datasets harmonisés: {sum(len(dfs) for dfs in harmonized_dataframes)}.')

2024-10-24 15:51:13,003 - INFO - caracteristiques: 4 datasets harmonisés.
2024-10-24 15:51:13,005 - INFO - lieux: 4 datasets harmonisés.
2024-10-24 15:51:13,005 - INFO - usagers: 4 datasets harmonisés.
2024-10-24 15:51:13,005 - INFO - vehicules: 4 datasets harmonisés.
2024-10-24 15:51:13,005 - INFO - Total datasets harmonisés: 16.


In [24]:
# Affichage des premières lignes des dataframes harmonisés
for group in harmonized_dataframes:
    for df_dict in group:
        for file_name, df in df_dict.items():
            print(f"Head of harmonized dataframe {file_name}:")
            display(df.head())

Head of harmonized dataframe caracteristiques-2019.csv:


,Num_Acc,jour,mois,an,hrmn,lum,dep,com,agg,int,atm,col
0,201900000001,30,11,2019,01::30,4,93,93053,1,1,1,2
1,201900000002,30,11,2019,02::50,3,93,93066,1,1,1,6
2,201900000003,28,11,2019,15::15,1,92,92036,1,1,1,4
3,201900000004,30,11,2019,20::20,5,94,94069,1,1,1,4
4,201900000005,30,11,2019,04::00,3,94,94028,1,1,1,2


Head of harmonized dataframe caracteristiques-2020.csv:


,Num_Acc,jour,mois,an,hrmn,lum,dep,com,agg,int,atm,col
0,202000000001,7,3,2020,16::55,1,91,91657,2,3,1,3
1,202000000002,7,3,2020,08::35,2,91,91657,2,9,7,6
2,202000000003,7,3,2020,13::30,1,91,91174,2,2,1,3
3,202000000004,7,3,2020,18::50,5,91,91215,2,1,1,6
4,202000000005,7,3,2020,11::00,1,77,77181,1,6,1,2


Head of harmonized dataframe carcteristiques-2021.csv:


...

,Num_Acc,id_vehicule,num_veh,senc,catv,obs,obsm,choc,manv,motor
0,202000000001,154 742 274,B01,1,7,0,2,2,15,1
1,202000000001,154 742 275,A01,1,33,0,2,1,2,1
2,202000000002,154 742 273,A01,3,7,0,1,2,26,1
3,202000000003,154 742 271,B01,1,7,0,2,1,1,1
4,202000000003,154 742 272,A01,1,7,0,2,8,1,0


Head of harmonized dataframe vehicules-2021.csv:


,Num_Acc,id_vehicule,num_veh,senc,catv,obs,obsm,choc,manv,motor
0,202100000001,201 764,B01,1,1,0,2,1,1,5
1,202100000001,201 765,A01,1,7,0,9,3,17,1
2,202100000002,201 762,A01,0,7,2,2,1,1,0
3,202100000002,201 763,B01,0,7,0,2,1,9,0
4,202100000003,201 761,A01,1,7,0,1,3,1,1


Head of harmonized dataframe vehicules-2022.csv:


,Num_Acc,id_vehicule,num_veh,senc,catv,obs,obsm,choc,manv,motor
0,202200000001,813 952,A01,1,2,0,2,1,9,1
1,202200000001,813 953,B01,1,7,0,2,2,1,1
2,202200000002,813 950,B01,2,7,0,2,8,15,1
3,202200000002,813 951,A01,2,10,0,2,1,1,1
4,202200000003,813 948,A01,2,7,0,2,1,2,1


In [25]:
# Comparaison des dataframes harmonisés aux structures de référence
compare_structures(harmonized_dataframes, reference_structures)

2024-10-24 15:51:13,146 - INFO - Comparaison pour le fichier caracteristiques-2019.csv:
+--------------------------+------------+
| Type de différence       | Colonnes   |
+==========================+============+
| Colonnes manquantes      |            |
+--------------------------+------------+
| Colonnes supplémentaires |            |
+--------------------------+------------+
| Différences de types     |            |
+--------------------------+------------+
2024-10-24 15:51:13,147 - INFO - Comparaison pour le fichier caracteristiques-2020.csv:
+--------------------------+------------+
| Type de différence       | Colonnes   |
+==========================+============+
| Colonnes manquantes      |            |
+--------------------------+------------+
| Colonnes supplémentaires |            |
+--------------------------+------------+
| Différences de types     |            |
+--------------------------+------------+
2024-10-24 15:51:13,148 - INFO - Comparaison pour le fichier carcter

In [26]:
# Listing des dataframes harmonisés
concatenated_dataframes = concat_harmonized_dataframes(dataframes)

# Affichage des dimensions des dataframes concaténés
for prefix, df in zip(prefixes, concatenated_dataframes):
    logging.info(f"{prefix}: {df.shape}")

2024-10-24 15:51:13,325 - INFO - caracteristiques: (218404, 12)
2024-10-24 15:51:13,326 - INFO - lieux: (218404, 14)
2024-10-24 15:51:13,327 - INFO - usagers: (494182, 16)
2024-10-24 15:51:13,331 - INFO - vehicules: (373584, 10)


## Fusion des dataframes harmonisés

In [28]:
# Vérification des dataframes concaténés après leur fusion
final_merged_df = merge_dataframes(concatenated_dataframes)

# Afficher les premiers et derniers enregistrements du DataFrame final pour vérification
logging.info(f"DataFrame final fusionné: {final_merged_df.shape}")
display(final_merged_df.head())
display(final_merged_df.tail())

2024-10-24 15:51:14,335 - INFO - DataFrame final fusionné: (494182, 47)


,Num_Acc,jour,mois,an,hrmn,lum,dep,com,agg,int,...,actp,etatp,id_usager,senc,catv,obs,obsm,choc,manv,motor
0,201900000001,30,11,2019,01::30,4,93,93053,1,1,...,-1,-1,NaN,2,7,0,2,5,23,1
1,201900000001,30,11,2019,01::30,4,93,93053,1,1,...,-1,-1,NaN,2,7,0,2,5,23,1
2,201900000001,30,11,2019,01::30,4,93,93053,1,1,...,-1,-1,NaN,2,17,1,0,3,11,1
3,201900000002,30,11,2019,02::50,3,93,93066,1,1,...,-1,-1,NaN,1,7,4,0,1,0,1
4,201900000003,28,11,2019,15::15,1,92,92036,1,1,...,0,-1,NaN,1,7,0,2,1,2,1


,Num_Acc,jour,mois,an,hrmn,lum,dep,com,agg,int,...,actp,etatp,id_usager,senc,catv,obs,obsm,choc,manv,motor
494177,202200055301,1,1,2022,08::40,1,81,81099,1,3,...,0,-1,968 230,2,7,0,0,8,19,1
494178,202200055301,1,1,2022,08::40,1,81,81099,1,3,...,0,-1,968 231,2,7,0,0,8,19,1
494179,202200055301,1,1,2022,08::40,1,81,81099,1,3,...,0,-1,968 232,2,7,0,2,1,1,1
494180,202200055302,1,3,2022,16::55,1,41,41018,2,1,...,-1,-1,968 228,1,33,0,2,1,1,1
494181,202200055302,1,3,2022,16::55,1,41,41018,2,1,...,-1,-1,968 229,1,7,0,0,4,21,1


In [29]:
final_merged_df['vma'].isna().sum()

0

## Contrôles après fusion

In [31]:
# Vérifier la présence de NaN dans le DataFrame final


def check_nan_presence(df):
    nan_columns = df.columns[df.isna().any()].tolist()
    if nan_columns:
        nan_proportions = df[nan_columns].isna().mean() * 100
        for col, prop in nan_proportions.items():
            logging.info(f"Colonne '{col}' contient {prop:.2f}% de NaN.")
    else:
        logging.info("Aucune colonne ne contient de NaN.")

check_nan_presence(final_merged_df)

2024-10-24 15:51:14,525 - INFO - Colonne 'an_nais' contient 1.20% de NaN.
2024-10-24 15:51:14,525 - INFO - Colonne 'id_usager' contient 48.22% de NaN.


In [32]:
# Vérifier le nombre de valeurs uniques dans chaque colonne après fusion.

def check_nunique(df):
    unique_proportions = df.nunique()
    for col, prop in unique_proportions.items():
        logging.info(f"Colonne '{col}' a {prop} valeurs uniques.")

check_nunique(final_merged_df)

2024-10-24 15:51:14,895 - INFO - Colonne 'Num_Acc' a 218404 valeurs uniques.
2024-10-24 15:51:14,895 - INFO - Colonne 'jour' a 31 valeurs uniques.
2024-10-24 15:51:14,895 - INFO - Colonne 'mois' a 12 valeurs uniques.
2024-10-24 15:51:14,895 - INFO - Colonne 'an' a 4 valeurs uniques.
2024-10-24 15:51:14,895 - INFO - Colonne 'hrmn' a 1438 valeurs uniques.
2024-10-24 15:51:14,895 - INFO - Colonne 'lum' a 6 valeurs uniques.
2024-10-24 15:51:14,895 - INFO - Colonne 'dep' a 116 valeurs uniques.
2024-10-24 15:51:14,895 - INFO - Colonne 'com' a 21259 valeurs uniques.
2024-10-24 15:51:14,895 - INFO - Colonne 'agg' a 2 valeurs uniques.
2024-10-24 15:51:14,895 - INFO - Colonne 'int' a 10 valeurs uniques.
2024-10-24 15:51:14,895 - INFO - Colonne 'atm' a 10 valeurs uniques.
2024-10-24 15:51:14,911 - INFO - Colonne 'col' a 8 valeurs uniques.
2024-10-24 15:51:14,912 - INFO - Colonne 'catr' a 8 valeurs uniques.
2024-10-24 15:51:14,912 - INFO - Colonne 'circ' a 5 valeurs uniques.
2024-10-24 15:51:14,91

## Fonctions de prétraitement complémentaire

In [34]:
# Prétraitement après fusion

def preprocessing_final_dataframe(df):
    # Supprimer les colonnes
    df = df.drop(columns=['id_usager', 'Num_Acc', 'com', 'id_vehicule', 'num_veh','lartpc'])
    
    # Modifier la colonne 'hrmn' en 'hour' et la passer du format HH:MM au format HH
    if 'hrmn' in df.columns:
        df['hour'] = df['hrmn'].str[:2].astype(int)
        df = df.drop(columns=['hrmn'])
    
    # Remplacer les valeurs manquantes de 'an_nais' par le mode de la colonne
    if 'an_nais' in df.columns:
        mode_an_nais = df['an_nais'].mode()[0]
        df['an_nais'] = df['an_nais'].fillna(mode_an_nais).astype(int)

    return df


# Encodage du dataframe

def encode_dataframe(df):
    dummy_columns = ['lum', 'agg', 'int', 'atm', 'col', 'catr', 'circ', 'prof', 'place', 'catu', 'sexe', 
                     'trajet', 'secu1', 'secu2', 'secu3', 'locp', 'actp', 'etatp', 'senc', 'catv', 'obs', 
                     'obsm', 'choc', 'manv', 'motor', 'plan', 'surf','an','infra','dep','situ','vosp']
    df = pd.get_dummies(df, columns=dummy_columns, drop_first= True)

    return df

## Lancement du processus final

In [36]:
final_merged_df = preprocessing_final_dataframe(final_merged_df)

In [37]:
# Vérifier le min et max de la colonne 'hour'
min_hour = final_merged_df['hour'].min()
max_hour = final_merged_df['hour'].max()
print(f"Min hour: {min_hour}, Max hour: {max_hour}")

Min hour: 0, Max hour: 23


In [38]:
final_merged_df = encode_dataframe(final_merged_df)

In [39]:
# Repérage des colonnes 'objet' restantes
object_columns = final_merged_df.select_dtypes(exclude=['int64', 'float64']).columns
print("Colonnes objet restantes:", object_columns)
print("Shape du dataframe avant conversion:", final_merged_df.shape)

# Boucle de modification des colonnes 'objet' restantes
for column in object_columns:
    try:
        final_merged_df[column] = final_merged_df[column].astype('int64')
    except ValueError:
        final_merged_df = final_merged_df[pd.to_numeric(final_merged_df[column], errors='coerce').notnull()]
        final_merged_df[column] = final_merged_df[column].astype('int64')

# Affichage de la nouvelle dimension du dataframe
print("Nouveau shape du dataframe:", final_merged_df.shape)
display(final_merged_df.columns)

# Chemin de sauvegarde CSV
PATH_TO_CSVS_PROCESSED = 'data/processed/data_init.csv'
final_merged_df.to_csv(PATH_TO_CSVS_PROCESSED)

Colonnes objet restantes: Index(['nbv', 'pr', 'pr1', 'an_nais', 'hour', 'lum_1', 'lum_2', 'lum_3',
       'lum_4', 'lum_5',
       ...
       'situ_2', 'situ_3', 'situ_4', 'situ_5', 'situ_6', 'situ_8', 'vosp_0',
       'vosp_1', 'vosp_2', 'vosp_3'],
      dtype='object', length=383)
Shape du dataframe avant conversion: (494182, 387)
Nouveau shape du dataframe: (374319, 387)


Index(['jour', 'mois', 'nbv', 'pr', 'pr1', 'vma', 'grav', 'an_nais', 'hour',
       'lum_1',
       ...
       'situ_2', 'situ_3', 'situ_4', 'situ_5', 'situ_6', 'situ_8', 'vosp_0',
       'vosp_1', 'vosp_2', 'vosp_3'],
      dtype='object', length=387)